In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# ⏳ The Two-Period Consumption Model

How do individuals make decisions about consumption and saving over time? The two-period consumption model provides a fundamental framework for analyzing this **intertemporal choice**.

Individuals aim to maximize their lifetime utility by choosing consumption levels in the present (Period 1, $c_1$) and the future (Period 2, $c_2$), subject to their lifetime income and the market interest rate.

This simulation visualizes the key components of this choice: the budget constraint, the consumer's preferences (indifference curves), and the optimality condition (Euler equation).

# ⚙️ Model Setup: Budget Constraint & Preferences

1.  **Intertemporal Budget Constraint (IBC):** This constraint states that the present value of lifetime consumption must equal the present value of lifetime income (wealth, $W$).
    $$ c_1 + \frac{c_2}{1 + r} = y_1 + \frac{y_2}{1 + r} \equiv W $$
    - $c_1, c_2$: Consumption in period 1 and 2.
    - $y_1, y_2$: Income (exogenous endowment) in period 1 and 2.
    - $r$: Real interest rate between period 1 and 2.
    * The slope of the budget line in the $(c_1, c_2)$ space is $-(1+r)$, representing the market trade-off between present and future consumption.

2.  **Preferences (Utility Function):** Individuals derive utility from consumption in both periods. We use a time-separable utility function, often logarithmic, incorporating a discount factor $\beta$ (beta):
    $$ U(c_1, c_2) = \ln(c_1) + \beta \ln(c_2) $$
    - $\beta$: Subjective discount factor ($0 < \beta \le 1$). It reflects how much the individual values future consumption relative to present consumption. $\beta=1$ means no subjective discounting, $\beta<1$ means future consumption is valued less.
    * Indifference curves represent combinations of $(c_1, c_2)$ yielding the same utility level. They are convex due to diminishing marginal utility.

# ✨ Optimal Choice: Euler Equation & Graphical Solution

The optimal consumption bundle $(c_1^*, c_2^*)$ maximizes utility subject to the budget constraint. This occurs where the **indifference curve is tangent to the budget line**.

**Optimality Condition (Tangency):**
$$ MRS = 1 + r $$
Where MRS is the Marginal Rate of Substitution between $c_1$ and $c_2$. For the utility function $U = \ln(c_1) + \beta \ln(c_2)$:
$$ MRS = \frac{\partial U / \partial c_1}{\partial U / \partial c_2} = \frac{1/c_1}{\beta (1/c_2)} = \frac{c_2}{\beta c_1} $$
Setting MRS equal to $(1+r)$:
$$ \frac{c_2}{\beta c_1} = 1 + r $$
$$ c_2 = \beta (1 + r) c_1 $$

This last equation is the **Euler Equation** for this specific utility function. It describes the optimal relationship between consumption in the two periods. It states that consumption should grow at a rate determined by the interest rate $r$ and the discount factor $\beta$.

**Graphical Interpretation:**
* **Budget Line:** Shows affordable consumption bundles.
* **Indifference Curves:** Show preferences (higher curves = higher utility).
* **Euler Line:** The line $c_2 = \beta (1 + r) c_1$ passes through the origin and represents all points satisfying the MRS = (1+r) condition.
* **Optimal Point $(c_1^*, c_2^*)$:** The single point where the Budget Line, the highest attainable Indifference Curve, and the Euler Line *all intersect*.

The simulation plots these three lines/curves and identifies the optimal consumption bundle.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def two_period_consumption_euler(y1=50.0, y2=50.0, r=0.05, beta=0.96, view_indiff=True):
    """
    Calculates and plots the optimal two-period consumption choice,
    highlighting the budget line, Euler line, and indifference curve.

    Args:
        y1 (float): Income in period 1.
        y2 (float): Income in period 2.
        r (float): Real interest rate.
        beta (float): Subjective discount factor (0 < beta <= 1).
        view_indiff (bool): Whether to plot indifference curves.
    """
    # Input validation
    y1 = max(y1, 0)
    y2 = max(y2, 0)
    r = max(r, -0.99) # Avoid r <= -1
    beta = np.clip(beta, 0.01, 1.0) # Ensure beta is valid

    # --- Calculations ---
    # Present Value of Lifetime Income (Wealth W)
    pv_income = y1 + y2 / (1 + r)

    # Optimal Consumption (derived from budget and Euler eq)
    # c1 + c2/(1+r) = W
    # c2 = beta*(1+r)*c1
    # c1 + beta*(1+r)*c1 / (1+r) = W
    # c1 * (1 + beta) = W
    if (1 + beta) < 1e-9: # Avoid division by zero if beta is somehow -1
        c1_star = 1e-9
    else:
        c1_star = pv_income / (1 + beta)

    # Ensure c1* is feasible (cannot exceed PV income) and non-negative
    c1_star = np.clip(c1_star, 1e-9, pv_income - 1e-9 if pv_income > 1e-6 else 1e-9)
    # Calculate c2* from Euler equation
    c2_star = beta * (1 + r) * c1_star
    c2_star = max(c2_star, 1e-9) # Ensure non-negative

    # Saving in period 1
    saving_s = y1 - c1_star

    # Utility function (handle log(0))
    def utility(c, l):
        c_safe = np.maximum(c, 1e-9)
        l_safe = np.maximum(l, 1e-9)
        with warnings.catch_warnings():
             warnings.simplefilter("ignore", category=RuntimeWarning) # Ignore log(0) warnings
             return np.log(c_safe) + beta * np.log(l_safe)

    # Utility level at optimum
    U_star = utility(c1_star, c2_star)

    # --- Calculations for Plotting ---
    # Define range for c1 (avoiding zero for logs/Euler line)
    c1_max_possible = pv_income # Max c1 if c2=0
    c1_vals = np.linspace(1e-3, c1_max_possible * 1.1, 400)

    # 1. Budget Line: c2 = (W - c1) * (1 + r)
    c2_budget = (pv_income - c1_vals) * (1 + r)
    # Filter out negative c2 for plotting
    c1_plot_budget = c1_vals[c2_budget >= 0]
    c2_plot_budget = c2_budget[c2_budget >= 0]

    # 2. Euler Line: c2 = beta * (1 + r) * c1
    c2_euler = beta * (1 + r) * c1_vals

    # 3. Indifference Curve at Optimum: c2 = exp( (U_star - log(c1)) / beta )
    with np.errstate(invalid='ignore', divide='ignore'): # Ignore log(0) warnings
        log_c1_vals = np.log(c1_vals)
    log_c2_indiff = (U_star - log_c1_vals) / beta
    # Clip to avoid extreme values before exponentiating
    log_c2_indiff = np.clip(log_c2_indiff, -30, 30)
    c2_indiff_star = np.exp(log_c2_indiff)


    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(9, 7))

    # Plot Budget Line
    ax.plot(c1_plot_budget, c2_plot_budget, label=f'Budget Line (W={pv_income:.1f})', color='black', linewidth=2)

    # Plot Euler Line
    ax.plot(c1_vals, c2_euler, label=f'Euler Line (Slope={beta*(1+r):.2f})', color='blue', linestyle=':', linewidth=2)

    # Plot Indifference Curve(s)
    if view_indiff:
        # Plot optimal indifference curve
        ax.plot(c1_vals, c2_indiff_star, linestyle='--', color='red', alpha=0.8, label=f'Optimal Indiff. Curve (U≈{U_star:.2f})')
        # Optionally plot nearby curves
        for offset in [-0.4, 0.4]:
             log_c2_nearby = (U_star + offset - log_c1_vals) / beta
             log_c2_nearby = np.clip(log_c2_nearby, -30, 30)
             c2_nearby = np.exp(log_c2_nearby)
             ax.plot(c1_vals, c2_nearby, linestyle=':', color='grey', alpha=0.5)

    # Plot Optimal Point (Intersection)
    ax.scatter(c1_star, c2_star, color='red', s=120, zorder=5, label='Optimal Choice (c1*, c2*)')
    ax.annotate(f"Optimal\nc1*={c1_star:.1f}\nc2*={c2_star:.1f}",
                xy=(c1_star, c2_star), xytext=(20, 20), textcoords='offset points',
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.2'),
                fontsize=10, bbox=dict(boxstyle='round,pad=0.3', fc='white', alpha=0.7))

    # Plot Endowment Point
    ax.scatter(y1, y2, color='green', marker='X', s=100, zorder=5, label=f'Endowment (y1={y1:.0f}, y2={y2:.0f})')


    ax.set_xlabel("Consumption Period 1 (c1)")
    ax.set_ylabel("Consumption Period 2 (c2)")
    ax.set_title("Two-Period Consumption Model: Optimal Choice")
    # Dynamic limits based on relevant points
    max_c1_axis = max(c1_max_possible, y1, c1_star) * 1.1
    max_c2_axis = max(c2_budget.max() if len(c2_plot_budget)>0 else 0, y2, c2_star, c2_euler.max()) * 1.1
    ax.set_xlim(0, max_c1_axis)
    ax.set_ylim(0, max_c2_axis)
    ax.legend(loc='best', fontsize='small')
    ax.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    # --- Display Results ---
    saving_status = "Saving" if saving_s > 1e-6 else ("Borrowing" if saving_s < -1e-6 else "Neither")
    results_md = f"""
    ### 📊 Optimal Consumption & Saving:

    * **Present Value of Income (W):** {pv_income:.2f}
    * **Optimal Consumption (c1\*):** {c1_star:.2f}
    * **Optimal Consumption (c2\*):** {c2_star:.2f}
    * **Saving in Period 1 (s = y1 - c1\*):** {saving_s:.2f} **({saving_status})**
    * **Utility Level at Optimum (U\*):** {U_star:.3f}
    """
    display(Markdown(results_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
layout = Layout(width='95%')

interact(
    two_period_consumption_euler,
    y1=FloatSlider(value=50.0, min=0.0, max=150.0, step=2.0, description='Income y1:', style=style, layout=layout),
    y2=FloatSlider(value=50.0, min=0.0, max=150.0, step=2.0, description='Income y2:', style=style, layout=layout),
    r=FloatSlider(value=0.05, min=-0.1, max=0.25, step=0.01, description='Interest Rate (r):', style=style, layout=layout, readout_format='.1%'),
    beta=FloatSlider(value=0.96, min=0.8, max=1.0, step=0.01, description='Discount Factor (beta β):', style=style, layout=layout, readout_format='.2f'),
    view_indiff=Dropdown(options=[True, False], value=True, description='Show Indiff Curves:', style=style, layout=layout),
);


<>:144: SyntaxWarning: invalid escape sequence '\*'
<>:144: SyntaxWarning: invalid escape sequence '\*'
<>:144: SyntaxWarning: invalid escape sequence '\*'
<>:144: SyntaxWarning: invalid escape sequence '\*'
<>:144: SyntaxWarning: invalid escape sequence '\*'
<>:144: SyntaxWarning: invalid escape sequence '\*'
<>:144: SyntaxWarning: invalid escape sequence '\*'
<>:144: SyntaxWarning: invalid escape sequence '\*'
/var/folders/kz/cc2y6j3953z1bgkfgj5xk9yc0000gn/T/ipykernel_87811/1551809298.py:144: SyntaxWarning: invalid escape sequence '\*'
  display(Markdown(results_md))
/var/folders/kz/cc2y6j3953z1bgkfgj5xk9yc0000gn/T/ipykernel_87811/1551809298.py:144: SyntaxWarning: invalid escape sequence '\*'
  display(Markdown(results_md))
/var/folders/kz/cc2y6j3953z1bgkfgj5xk9yc0000gn/T/ipykernel_87811/1551809298.py:144: SyntaxWarning: invalid escape sequence '\*'
  display(Markdown(results_md))
/var/folders/kz/cc2y6j3953z1bgkfgj5xk9yc0000gn/T/ipykernel_87811/1551809298.py:144: SyntaxWarning: inval

interactive(children=(FloatSlider(value=50.0, description='Income y1:', layout=Layout(width='95%'), max=150.0,…